In [71]:
import numpy as np
import torch 

import pennylane as qml
from pennylane.operation import Operation, AnyWires
import matplotlib.pyplot as plt
import pennylane.numpy as np
import numpy as np
import pandas as pd
from sklearn.datasets import *
import wandb
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

from tqdm import tqdm
dataset_list={"iris": 1,"digits": 2,"wine": 3,"cancer": 4, "iris_linear": 5, "moon": 6}
s=0.05
init_method=lambda x: torch.nn.init.uniform_(x,a=0.,b=s*np.pi)

In [72]:
def stereo_pj(X):
    n,m=np.shape(X)
    newX=np.zeros((n,m+1))
    for rowindex,x in enumerate(X):
        s=np.sum(pow(x,2))
        for index in range(m):
            newX[rowindex,index]=2*x[index]/(s+1)
        newX[rowindex,m]=(s-1)/(s+1)
    return newX

In [73]:
def get_dataset(index, split=True, split_percentage=0.33, standardization_mode=1):
    """
    iris:          Load and return the iris dataset (classification).
    digits:        Load and return the digits dataset (classification).
    wine:          Load and return the wine dataset (classification).
    breast_cancer: Load and return the breast cancer wisconsin dataset (classification).
    """
    X=None
    y=None
    n_classes=None
    print("Getting dataset: ",{i for i in dataset_list if dataset_list[i]==index})
    match index:
        case 1:
            dataset=load_iris()
            X = dataset.data
            y = dataset.target
            n_classes=len(np.unique(y))
        case 2:
            dataset=load_digits()#TODO: PCA
            X = dataset.data
            y = dataset.target
            n_classes=len(np.unique(y))
        case 3:
            dataset=load_wine()
            X = dataset.data
            y = dataset.target
            n_classes=len(np.unique(y))
        case 4:
            dataset=load_breast_cancer()
            X = dataset.data
            y = dataset.target
            n_classes=len(np.unique(y))
        case 5:
            iris_data = load_iris()
            iris_df = pd.DataFrame(data=iris_data['data'], columns=iris_data['feature_names'])
            
            iris_df['Iris type'] = iris_data['target']
            iris_df = iris_df[iris_df["Iris type"] != 2]
            iris = iris_df.to_numpy()
            
            X = iris[:, :4]  # we only take the first two features.
            y = np.ndarray.tolist(iris[:, 4].astype(int))
            n_classes=len(np.unique(y))
        case 6:
            X, y = make_moons(n_samples=200, noise=0.1)
            n_classes=len(np.unique(y))
        case _:
            raise Exception("Sorry, the dataset does not exist")


    match standardization_mode:
        case 1:
            X_mean, X_std=np.mean(X,axis=0), np.std(X,axis=0,ddof=1)
            
            X=(X-X_mean)/X_std
            X=np.hstack((X,2.0*np.ones(X.shape[0])[:,None]))
            X=np.array([np.clip(row/np.sqrt(np.sum(row**2)),-1,1) for row in X])
        case 2:
            X=stereo_pj(X)

    y_hot=torch.zeros((len(y),n_classes),requires_grad=False)
    for index,element in enumerate(y):
        y_hot[index][element]=1

    if split:
        return train_test_split(X, y_hot, test_size=split_percentage, random_state=42)
    else:
        return X, y_hot


In [74]:
class RBSGate(Operation):
    num_wires = 2  

    def __init__(self, theta, wires, id=None):
        all_wires = qml.wires.Wires(wires)
        super().__init__(theta, wires=all_wires, id=id)

    @staticmethod
    def compute_decomposition(theta, wires):
        decomp = [
                qml.Hadamard(wires=wires[0]),
                qml.Hadamard(wires=wires[1]),
                qml.CZ(wires=wires),
                qml.RY(theta/2.,wires=wires[0]),
                qml.RY(-theta/2.,wires=wires[1]),
                qml.CZ(wires=wires),
                qml.Hadamard(wires=wires[0]),
                qml.Hadamard(wires=wires[1])
            ]
        return decomp

In [75]:
X,Y=get_dataset(5,split=False)

Getting dataset:  {'iris_linear'}


In [76]:
def probs_single1(inputs, weights):
    shape=[3,5]
    max_q=np.max(shape)
    
    q_base=max_q-shape[0]
    qml.PauliX(wires=q_base)

    prd_fact=1.0
    for qi_idx, qi in enumerate(range(max_q-shape[0],max_q-1)):

        theta_i=torch.arccos((inputs[...,qi_idx])/prd_fact)
        prd_fact=prd_fact*torch.sin(theta_i)
        RBSGate(theta_i,wires=[qi,qi+1],id=f"$\\alpha1_{{{{{qi}}}}}$")

    ctr=0
    
    for ji,j in enumerate(range(max_q,max_q-shape[1],-1)):
        for i in range(q_base-1-ji,q_base-1-ji+shape[0]):
            if i<0:
                continue
            RBSGate(weights[0][ctr],[i,i+1],id=f"$\\theta1_{{{{{ctr}}}}}$")
            ctr+=1
    return qml.probs(wires=range(max_q-shape[1],max_q))


In [77]:
class ProbsToUnaryLayer(torch.nn.Module):

    def __init__(self, size_in):
        super(ProbsToUnaryLayer, self).__init__()
        self.size_q_in=size_in

    def forward(self, input_var):
       # print("probstounitary")
        #print(input_var)

        filt = [2**i for i in range(self.size_q_in)]
        #print("filtered: ",input_var[:, filt])
        #input()
        return input_var[:, filt]*12-6

In [78]:
class QPNN:
    def __init__(self,structure,X,y,xval=None,yval=None):
        self.X=X
        self.y=y
        self.xval=xval
        self.yval=yval
        self.architecture=[np.shape(X)[1]]+structure+[int((np.shape(y)[1]))]
        print(self.architecture)
        self.nqubits=np.max(self.architecture)
        print(self.nqubits)
        self.cuda=torch.cuda.is_available()
        self.devs=[]
        self.qnodes=[]
        self.qlayers=[]
        self.model_architecture=[]
        for layer in range(len(self.architecture)-1):
            n_layers = 1
            n_pars = int((2*self.architecture[layer]-1-self.architecture[layer+1])*(self.architecture[layer+1])/2)
            
            dev = qml.device("default.qubit", wires=self.nqubits) #TODO: ADD CUDA
            qnode = qml.QNode(self.probs_single, dev)
            weight_shapes = {"weights": (n_layers, n_pars),"weights_aux":(self.architecture[layer],self.architecture[layer+1])}
            qlayer = qml.qnn.TorchLayer(qnode, weight_shapes,init_method=init_method)#
            self.devs.append(dev)
            self.qnodes.append(qnode)
            self.qlayers.append(qlayer)
            self.model_architecture.append(qlayer)
            self.model_architecture.append(ProbsToUnaryLayer(self.architecture[layer+1]))
            self.model_architecture.append(torch.nn.Softmax(dim=1))
        
        self.model= torch.nn.Sequential(*self.model_architecture)
        for index,x in  enumerate(reversed(list(self.model.parameters()))):
                if index%2==0:
                    x.requires_grad =False    
        
    def train(self):
        opt = torch.optim.SGD(self.model.parameters(), lr=0.1)

        loss = torch.nn.CrossEntropyLoss()
        
        X = torch.tensor(self.X, requires_grad=False).float()
        y = torch.tensor(self.y, requires_grad=False).float()
        if self.xval is not None:
            Xval = torch.tensor(self.xval, requires_grad=False).float()
            yval = torch.tensor(self.yval, requires_grad=False).float()

        batch_size = 1
        batches = 200 // batch_size

        data_loader = torch.utils.data.DataLoader(
            list(zip(X, y)), batch_size=batch_size, shuffle=True, drop_last=True
        )
        
        epochs = 50
        print([pm for pm in self.model.parameters()])
        for epoch in tqdm(range(epochs)):
        
            running_loss = 0
            
            for xs, ys in data_loader:
                opt.zero_grad()
                res=self.model(xs)
                loss_evaluated = loss(res, ys)

                loss_evaluated.backward()
        
                opt.step()
        
                running_loss += loss_evaluated
        
            avg_loss = running_loss / batches
            #print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))
        print([pm for pm in self.model.parameters()])
        y_pred = self.model(X)
        predictions = torch.argmax(y_pred, axis=1).detach().numpy()
        truelabel=torch.argmax(y, axis=1).detach().numpy()
        #print(predictions)
        #print(y)
        correct = [1 if p == p_true else 0 for p, p_true in zip(predictions, truelabel)]
        accuracy = sum(correct) / len(correct)
        print(f"Accuracy: {accuracy * 100}%")
        
        
        
    def probs_single(self,inputs, weights,weights_aux):
        shape=np.shape(weights_aux)
        max_q=np.max(shape)
        
        q_base=max_q-shape[0]
        qml.PauliX(wires=q_base)
        prd_fact=1.0
        for qi_idx, qi in enumerate(range(max_q-shape[0],max_q-1)):

            theta_i=torch.arccos((inputs[...,qi_idx])/prd_fact)
            prd_fact=prd_fact*torch.sin(theta_i)
            RBSGate(theta_i,wires=[qi,qi+1],id=f"$\\alpha1_{{{{{qi}}}}}$")

        ctr=0
        for ji,j in enumerate(range(max_q,max_q-shape[1],-1)):
            for i in range(q_base-1-ji,q_base-1-ji+shape[0]):
                if i<0:
                    continue
                RBSGate(weights[0][ctr],[i,i+1],id=f"$\\theta1_{{{{{ctr}}}}}$")
                ctr+=1
        return qml.probs(wires=range(max_q-shape[1],max_q))
        
        
        
        

In [79]:
net=QPNN([5],X,Y)
net.train()

C:\Users\giaco\AppData\Local\Temp\ipykernel_11576\1030324352.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.y, requires_grad=False).float()


[5, 5, 2]
5
[Parameter containing:
tensor([[0.1448, 0.1388, 0.0537, 0.0781, 0.0980, 0.0725, 0.0423, 0.1389, 0.1031,
         0.0828]], requires_grad=True), Parameter containing:
tensor([[0.1405, 0.1104, 0.1197, 0.0120, 0.1070],
        [0.1039, 0.0760, 0.1324, 0.0967, 0.0820],
        [0.0205, 0.0724, 0.0392, 0.0736, 0.1101],
        [0.0281, 0.1449, 0.1144, 0.0735, 0.0162],
        [0.0166, 0.0217, 0.1040, 0.0572, 0.0337]]), Parameter containing:
tensor([[0.1445, 0.0010, 0.0614, 0.0694, 0.1066, 0.0252, 0.0777]],
       requires_grad=True), Parameter containing:
tensor([[0.1488, 0.0004],
        [0.1156, 0.1401],
        [0.0124, 0.0881],
        [0.0370, 0.0812],
        [0.1009, 0.1507]])]


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [06:11<00:00,  7.42s/it]


[Parameter containing:
tensor([[ 0.0743,  2.2240,  0.2274, -1.6976,  0.8266,  1.0648, -1.2466,  0.5797,
          0.2586,  0.1843]], requires_grad=True), Parameter containing:
tensor([[0.1405, 0.1104, 0.1197, 0.0120, 0.1070],
        [0.1039, 0.0760, 0.1324, 0.0967, 0.0820],
        [0.0205, 0.0724, 0.0392, 0.0736, 0.1101],
        [0.0281, 0.1449, 0.1144, 0.0735, 0.0162],
        [0.0166, 0.0217, 0.1040, 0.0572, 0.0337]]), Parameter containing:
tensor([[ 1.6508,  1.5728,  3.8838, -2.3767,  0.3972, -1.0993, -1.8114]],
       requires_grad=True), Parameter containing:
tensor([[0.1488, 0.0004],
        [0.1156, 0.1401],
        [0.0124, 0.0881],
        [0.0370, 0.0812],
        [0.1009, 0.1507]])]
Accuracy: 100.0%
